In [1]:
import os
import glob
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='api.env',override=True)

True

In [3]:
openai_api_key=os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ali-onboarding-api-key')

In [4]:
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [5]:
MODEL = "gpt-4o-mini"

In [6]:
context = {}

employees = glob.glob("Knowledge-base/Employees/*")

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [7]:
context["Lancaster"]

"# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferenc

In [8]:
products = glob.glob("Knowledge-base/Products/*")

for product in products:
    name = product.split(os.sep)[-1][:-3]
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [9]:
context.keys()

dict_keys(['Chen', 'Spencer', 'Tran', 'Blake', 'Lancaster', 'Thompson', 'Greene', 'Thomson', 'Trenton', 'Harper', 'Bishop', 'Carter', 'Rellm', 'Markellm', 'Homellm', 'Carllm'])

In [10]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [11]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context          

In [12]:
get_relevant_context("Who is lancaster?")

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferen

In [13]:
get_relevant_context("Who is Avery and what is carllm?")

['# Product Summary\n\n# Carllm\n\n## Summary\n\nCarllm is an innovative auto insurance product developed by Insurellm, designed to streamline the way insurance companies offer coverage to their customers. Powered by cutting-edge artificial intelligence, Carllm utilizes advanced algorithms to deliver personalized auto insurance solutions, ensuring optimal coverage while minimizing costs. With a robust infrastructure that supports both B2B and B2C customers, Carllm redefines the auto insurance landscape and empowers insurance providers to enhance customer satisfaction and retention.\n\n## Features\n\n- **AI-Powered Risk Assessment**: Carllm leverages artificial intelligence to analyze driver behavior, vehicle conditions, and historical claims data. This enables insurers to make informed decisions and set competitive premiums that reflect true risk profiles.\n\n- **Instant Quoting**: With Carllm, insurance companies can offer near-instant quotes to customers, enhancing the customer exper

In [14]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [15]:
print(add_context("Who is Alex Lancaster?"))

Who is Alex Lancaster?

The following additional context might be relevant in answering this question:

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  

- **2010 - 2013**: Business Analyst at Edge Analytics  
  Prior to joining Innova

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    stream = client.chat.completions.create(model=MODEL, messages=messages, stream=True)


    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response


In [25]:
view = gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [26]:
# Test the chat function to ensure it works
test_history = []
test_message = "Who is Avery Lancaster?"

print("Testing chat function...")
try:
    for response in chat(test_message, test_history):
        print(f"Response: {response}")
        break  # Just get the first response for testing
    print("✅ Chat function is working correctly!")
except Exception as e:
    print(f"❌ Error in chat function: {e}")


Testing chat function...
Response: 
✅ Chat function is working correctly!


In [17]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [18]:
folders = glob.glob("Knowledge-base/")
text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [19]:
len(documents)

30

In [20]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [21]:
len(chunks)

119

In [22]:
chunks[26]

Document(metadata={'source': 'Knowledge-base/Contracts/Contract with Apex Reinsurance for Rellm.md', 'doc_type': ''}, page_content='# Contract with Apex Reinsurance for Rellm: AI-Powered Enterprise Reinsurance Solution\n\n## Terms\n\n1. **Parties Involved**: This contract (“Agreement”) is entered into between Insurellm, Inc. (“Provider”) and Apex Reinsurance (“Client”) on this [Date].\n\n2. **Scope of Services**: Provider agrees to deliver the Rellm solution, which includes AI-driven analytics, seamless integrations, risk assessment modules, customizable dashboards, regulatory compliance tools, and client and broker portals as described in the product summary.\n\n3. **Payment Terms**: Client shall pay the Provider the sum of $10,000 per month for the duration of this agreement. Payments are due on the first day of each month and will be processed via electronic funds transfer.\n\n4. **Contract Duration**: This Agreement shall commence on [Start Date] and shall remain in effect for a pe

In [23]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: 


In [24]:
for chunk in chunks:
    if 'CEO' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='## Support

1. **Customer Support**: Velocity Auto Solutions will have access to Insurellm’s customer support team via email or chatbot, available 24/7.  
2. **Technical Maintenance**: Regular maintenance and updates to the Carllm platform will be conducted by Insurellm, with any downtime communicated in advance.  
3. **Training & Resources**: Initial training sessions will be provided for Velocity Auto Solutions’ staff to ensure effective use of the Carllm suite. Regular resources and documentation will be made available online.

---

**Accepted and Agreed:**  
**For Velocity Auto Solutions**  
Signature: _____________________  
Name: John Doe  
Title: CEO  
Date: _____________________  

**For Insurellm**  
Signature: _____________________  
Name: Jane Smith  
Title: VP of Sales  
Date: _____________________' metadata={'source': 'Knowledge-base/Contracts/Contract with Velocity Auto Solutions for Carllm.md', 'doc_type': ''}
_________
page_content='3. **Regular Updates:**

In [ ]:

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import plotly.graph_objects as go


In [ ]:
class SimpleVectorStore:
    def __init__(self, documents, embeddings_model):
        self.documents = documents
        self.embeddings_model = embeddings_model
        self.embeddings = None
        self._build_embeddings()
    
    def _build_embeddings(self):
        """Create embeddings for all documents"""
        print("Creating embeddings for documents...")
        texts = [doc.page_content for doc in self.documents]
        self.embeddings = self.embeddings_model.embed_documents(texts)
        print(f"✅ Created embeddings for {len(self.documents)} documents")
    
    def similarity_search(self, query, k=3):
        """Search for similar documents using cosine similarity"""
        # Get query embedding
        query_embedding = self.embeddings_model.embed_query(query)
        query_embedding = np.array(query_embedding).reshape(1, -1)
        
        # Calculate cosine similarities
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        
        # Get top k most similar documents
        top_indices = np.argsort(similarities)[::-1][:k]
        
        results = []
        for idx in top_indices:
            results.append({
                'document': self.documents[idx],
                'similarity': similarities[idx]
            })
        
        return [result['document'] for result in results]


print("Creating simple vector store...")

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

vectorstore = SimpleVectorStore(chunks, embeddings)

print(f"✅ Vector store created with {len(chunks)} documents")
print(f"Vector store type: {type(vectorstore)}")


Creating simple vector store...
Creating embeddings for documents...
✅ Created embeddings for 119 documents
✅ Vector store created with 119 documents
Vector store type: <class '__main__.SimpleVectorStore'>


In [35]:

# Function to search for relevant documents
def search_documents(query, k=3):
    """
    Search for relevant documents using simple vector store
    
    Args:
        query (str): The search query
        k (int): Number of documents to return
    
    Returns:
        List of relevant documents
    """
    # Perform similarity search
    docs = vectorstore.similarity_search(query, k=k)
    
    print(f"Found {len(docs)} relevant documents for query: '{query}'")
    for i, doc in enumerate(docs, 1):
        print(f"\n--- Document {i} ---")
        print(f"Source: {doc.metadata.get('source', 'Unknown')}")
        print(f"Content preview: {doc.page_content[:200]}...")
    
    return docs

# Test the search function
test_query = "Who is Olivevr Spencer and what is her role?"
relevant_docs = search_documents(test_query, k=2)


Found 2 relevant documents for query: 'Who is Olivevr Spencer and what is her role?'

--- Document 1 ---
Source: Knowledge-base/Employees/Oliver Spencer.md
Content preview: # HR Record

# Oliver Spencer

## Summary
- **Date of Birth**: May 14, 1990  
- **Job Title**: Backend Software Engineer  
- **Location**: Austin, Texas  

## Insurellm Career Progression
- **March 20...

--- Document 2 ---
Source: Knowledge-base/Employees/Samantha Greene.md
Content preview: # Samantha Greene

## Summary
- **Date of Birth:** October 14, 1990
- **Job Title:** HR Generalist
- **Location:** Denver, Colorado

## Insurellm Career Progression
- **2020** - Joined Insurellm as a ...


In [36]:
# Enhanced chat function using simple vector store
def enhanced_chat(message, history):
    """
    Enhanced chat function that uses simple vector store for better context retrieval
    """
    # Search for relevant documents
    relevant_docs = vectorstore.similarity_search(message, k=3)
    
    # Build context from relevant documents
    context = ""
    for i, doc in enumerate(relevant_docs, 1):
        context += f"Document {i}:\n{doc.page_content}\n\n"
    
    # Create messages with system prompt and context
    messages = [{"role": "system", "content": f"{system_message}\n\nRelevant context:\n{context}"}] + history
    messages.append({"role": "user", "content": message})

    # Get response from OpenAI
    stream = client.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# Test the enhanced chat function
print("Testing enhanced chat function...")
test_history = []
test_message = "What products does Insurellm offer?"

print(f"Question: {test_message}")
print("Answer:")
for response in enhanced_chat(test_message, test_history):
    print(response, end="")
    break  # Just get the first response for testing


Testing enhanced chat function...
Question: What products does Insurellm offer?
Answer:


In [ ]:
print("Creating enhanced RAG interface...")

enhanced_view = gr.ChatInterface(
    enhanced_chat, 
    title="Insurellm RAG Assistant (Enhanced with Vector Search)",
    description="Ask questions about Insurellm employees, products, and contracts. Powered by semantic vector search.",
    type="messages"
).launch()


Creating enhanced RAG interface...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [38]:
# Simple persistence for vector store
import pickle
import os

def save_vectorstore(vectorstore, path="simple_vectorstore.pkl"):
    """Save simple vector store to disk"""
    try:
        with open(path, 'wb') as f:
            pickle.dump({
                'documents': vectorstore.documents,
                'embeddings': vectorstore.embeddings
            }, f)
        print(f"✅ Vector store saved to {path}")
    except Exception as e:
        print(f"❌ Error saving vector store: {e}")

def load_vectorstore(embeddings_model, path="simple_vectorstore.pkl"):
    """Load simple vector store from disk"""
    try:
        if os.path.exists(path):
            with open(path, 'rb') as f:
                data = pickle.load(f)
            vectorstore = SimpleVectorStore.__new__(SimpleVectorStore)
            vectorstore.documents = data['documents']
            vectorstore.embeddings = data['embeddings']
            vectorstore.embeddings_model = embeddings_model
            print(f"✅ Vector store loaded from {path}")
            return vectorstore
        else:
            print(f"❌ File {path} not found")
            return None
    except Exception as e:
        print(f"❌ Error loading vector store: {e}")
        return None

# Save the current vector store
save_vectorstore(vectorstore, "insurellm_vectorstore.pkl")


✅ Vector store saved to insurellm_vectorstore.pkl


ModuleNotFoundError: No module named 'chromadb'